In [ ]:
import selectors
import socket

sel = selectors.DefaultSelector()

def accept(sock, mask):
    conn, addr = sock.accept()  # Should be ready
    print('accepted', conn, 'from', addr)
    conn.setblocking(False)
    sel.register(conn, selectors.EVENT_READ, read)

def read(conn, mask):
    data = conn.recv(1000)  # Should be ready
    if data:
        print('echoing', repr(data), 'to', conn)
        conn.send(data)  # Hope it won't block
    else:
        print('closing', conn)
        sel.unregister(conn)
        conn.close()

sock = socket.socket()
sock.bind(('localhost', 122))
sock.listen(100)
sock.setblocking(False)
sel.register(sock, selectors.EVENT_READ, accept)

while True:
    events = sel.select()
    for key, mask in events:
        callback = key.data
        callback(key.fileobj, mask)

In [ ]:
import socket
import time

IP = '127.0.0.1'
PORT = 122
DELAY = 2  # 延迟时间
DATA = "Hello Socket"


def TcpClient():
    # 建立一个客户端
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # 声明socket类型，同时生成链接对象
    try:
        client.connect((IP, PORT))  # 建立一个链接，连接到本地的6969端口
    except ConnectionRefusedError:
        print("服务器无法连接，请确保服务器端已打开。")
        exit(0)
    while True:
        try:
            client.sendall(DATA.encode("utf-8"))
            print(DATA, "已发送")
            data = client.recv(1024)  # 接收一个信息，并指定接收的大小 为1024字节
            print("接收到:", data)
            time.sleep(DELAY)
        except ConnectionAbortedError:
            print("服务器无法连接，请确保服务器端已打开。")
            exit(0)
        except KeyboardInterrupt:
            client.close()
            print("Tcp 客户端已关闭")
            exit(0)


if __name__ == "__main__":
    TcpClient()